# Etude des oeuvres les plus empruntés en 2017 dans les bibliothèques parisiennes

Ce jeu de données provient du site du gouvernement : data.gouv.fr. Il contient des informations sur les livres les plus prêtés dans les bibliothèques parisiennes. On va donc s'intéresser au type d'ouvrage le plus empruntés pour voir quel type de public est le plus susceptible de venir emprunter des documents dans une bibliothèque.

## 1. Nettoyage des données

Avant de commencer notre analyse à proprement parler, on va tout d'abord commencer par nettoyer nos données et voir comment résoudre certaines anomalies que l'on pourrait rencontrer.

La première étape consiste à importer deux modules qui vont nous aider à faire ce nettoyage des données.

In [1]:
import numpy as np
import pandas as pd

Une fois les modules importés, on charge le fichier qui va être utilisé pour l'analyse. 

In [2]:
df = pd.read_csv('prets_2017.csv', sep=';')

On regarde ensuite diverses informations concernant notre fichier de données.

In [3]:
print(df.shape)
df.head(2)

(825, 9)


,rang,type_de_documnt,prets,titre,auteur,nombre_de_localisations,nombre_de_pret_total,nombre_de_prets_2017,nombre_d_exemplaires
0,1.0,Livre jeunesse,1189,Max se trouve trop petit,"Saint-Mars, Dominique de",53,3471,1799,135
1,11.0,Livre jeunesse,1019,Max et Lili ont peur des images violentes,"Saint-Mars, Dominique de",51,4522,1407,119


Notre fichier de données comporte 825 entrées correspondant chacun à un document. Il possède 9 colonnes qui correspondent aux diverses informations de notre fichier. Pour chaque document, on possède : 
- son rang (classement de l'oeuvre dans le fichier)
- son type (livre, DVD, etc.)
- son titre
- son auteur
- son nombre de localisations, c'est-à-dire le nombre de bibliothèques qui possède ce document
- son nombre de prêt total, qui est le nombre de prêt depuis que le document est présent
- son nombre de prêt pour l'année 2017
- son nombre d'exemplaires

### 1.1. Analyses des données manquantes

In [4]:
missing_values_count = df.isna().sum()
print(missing_values_count)

rang                        0
type_de_documnt             0
prets                       0
titre                       0
auteur                     37
nombre_de_localisations     0
nombre_de_pret_total        0
nombre_de_prets_2017        0
nombre_d_exemplaires        0
dtype: int64


La colonne 'auteur' comporte 37 entrées manquantes. Une fois ces entrées identifiées, il est intéressant de savoir pourquoi.

In [5]:
df = df.sort_values('auteur', ascending=True)
df.tail(37)

,rang,type_de_documnt,prets,titre,auteur,nombre_de_localisations,nombre_de_pret_total,nombre_de_prets_2017,nombre_d_exemplaires
11,373.0,Jeux de société,475,Uno :,NaN,3,522,35,7
24,630.0,DVD jeunesse,386,Les aristochats,NaN,40,1244,490,50
29,704.0,DVD jeunesse,368,Raiponce,NaN,40,3306,495,53
64,529.0,DVD jeunesse,413,L'âge de glace 4,NaN,37,3190,545,53
69,618.0,DVD jeunesse,388,BGG - Le Bon Gros Géant,NaN,39,873,389,42
110,441.0,DVD jeunesse,447,Rebelle,NaN,42,3101,562,59
114,505.0,Bande dessinée jeunesse,424,Game over. 6. Sound of silence,NaN,37,2567,564,45
116,521.0,Jeux Vidéos tous publics,417,Tablette Ipad : Parascolaire,NaN,3,707,266,7
155,325.0,Bande dessinée jeunesse,500,Les Schtroumpfs de l'ordre,NaN,38,4599,694,54
172,676.0,DVD jeunesse,374,La Colline aux coquelicots,NaN,36,3172,452,49


On remarque que la plupart des données n'ayant pas d'auteurs sont des DVD jeunesse. Cependant, il est possible de retrouver l'auteur pour les bandes dessinées jeunesse. 
- Game Over. 6. Sound of Silence : Midam
- Les Schtroumpfs de l'ordre : Peyo
- No problemo : Midam
- Mortelle Adèle. T10. Choudiboulove : Tan, Le Feyer

In [6]:
df.loc[114] = df.loc[114].fillna('Midam')
df.loc[155] = df.loc[155].fillna('Peyo')
df.loc[397] = df.loc[397].fillna('Midam')
df.loc[752] = df.loc[752].fillna('Tan')

In [7]:
missing_values_count

rang                        0
type_de_documnt             0
prets                       0
titre                       0
auteur                     37
nombre_de_localisations     0
nombre_de_pret_total        0
nombre_de_prets_2017        0
nombre_d_exemplaires        0
dtype: int64

Après ces ajouts, il reste toujours 33 valeurs manquantes correspondant à des DVD, des jeux vidéo ou des jeux de société. 

### 1.2. Analyses des données dupliquées

In [8]:
duplicated = df.duplicated()
df[duplicated]

,rang,type_de_documnt,prets,titre,auteur,nombre_de_localisations,nombre_de_pret_total,nombre_de_prets_2017,nombre_d_exemplaires


Notre fichier ne présente pas de données dupliquées. On peut donc passer à l'analyse exploratoire.

In [9]:
df.to_csv('prets_2017_corr.csv')

Pour ne pas modifier l'intégrité du fichier source, on crée un fichier .csv avec les différentes corrections effectués dans la première partie. 

## 2. Analyses Exploratoires des données

In [10]:
df = pd.read_csv('prets_2017_corr.csv')